In [ ]:
import yfinance as yf
import pandas as pd
import requests
import time
from tqdm import tqdm
import sqlite3

class YFinanceCompanyDatabase:
    def __init__(self):
        self.companies_df = None
        
    def get_sp500_tickers(self):
        """Get S&P 500 companies using reliable sources"""
        try:
            # Method 1: Direct download from S&P Global
            url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/main/data/constituents.csv"
            sp500_df = pd.read_csv(url)
            tickers = sp500_df['Symbol'].tolist()
            return tickers
        except Exception as e:
            print(f"❌ S&P Global method failed: {e}")
        
        try:
            # Method 2: Wikipedia fallback
            url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
            tables = pd.read_html(url)
            sp500_table = tables[0]
            tickers = sp500_table['Symbol'].tolist()
            return tickers
        except Exception as e:
            print(f"❌ Wikipedia failed: {e}")
        
        # Final fallback: Full S&P 500 list
        return self._get_full_sp500_fallback()
    
    def _get_full_sp500_fallback(self):
        """Full S&P 500 list as fallback"""
        full_sp500 = [
            'MMM', 'AOS', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP',
            'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALGN',
            'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMD', 'AEE',
            'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH',
            'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ANET',
            'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'AZO', 'AVB', 'AVY', 'BKR', 'BALL',
            'BAC', 'BBWI', 'BAX', 'BDX', 'WRB', 'BRK.B', 'BBY', 'BIO', 'TECH', 'BIIB',
            'BLK', 'BK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR',
            'BRO', 'BF.B', 'CHRW', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL',
            'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CDAY',
            'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF',
            'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL',
            'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA',
            'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA',
            'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DISCA', 'DISCK',
            'DISH', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DTE', 'DUK', 'DRE',
            'DD', 'DXC', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'EMR',
            'ENPH', 'ETR', 'EOG', 'EPAM', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY',
            'RE', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS',
            'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FE', 'FRC', 'FISV', 'FLT', 'FMC',
            'F', 'FTNT', 'FTV', 'FBHS', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT',
            'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GL', 'GS',
            'HAL', 'HBI', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT',
            'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUM', 'HBAN', 'HII',
            'IEX', 'IDXX', 'INFO', 'ITW', 'ILMN', 'INCY', 'IR', 'INTC', 'ICE', 'IBM',
            'IP', 'IPG', 'IFF', 'INTU', 'ISRG', 'IVZ', 'IPGP', 'IQV', 'IRM', 'JKHY',
            'J', 'JBHT', 'SJM', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KEY', 'KEYS',
            'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW',
            'LVS', 'LDOS', 'LEN', 'LLY', 'LNC', 'LIN', 'LYV', 'LKQ', 'LMT', 'L',
            'LOW', 'LUMN', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM',
            'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET',
            'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'TAP', 'MDLZ',
            'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX',
            'NWL', 'NEM', 'NWSA', 'NWS', 'NEE', 'NLSN', 'NKE', 'NI', 'NSC', 'NTRS',
            'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL',
            'OMC', 'OKE', 'ORCL', 'OGN', 'OTIS', 'PCAR', 'PKG', 'PH', 'PAYX', 'PAYC',
            'PYPL', 'PNR', 'PEP', 'PKI', 'PFE', 'PM', 'PSX', 'PNW', 'PXD', 'PNC',
            'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC',
            'PSA', 'PHM', 'PVH', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX',
            'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RHI', 'ROK', 'ROL', 'ROP',
            'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SEE', 'SRE', 'NOW',
            'SHW', 'SPG', 'SWKS', 'SNA', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STE',
            'SYK', 'SIVB', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TGT',
            'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO',
            'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'UDR', 'ULTA', 'USB',
            'UAA', 'UA', 'UNP', 'UAL', 'UNH', 'UPS', 'URI', 'UHS', 'VLO', 'VTR',
            'VRSN', 'VRSK', 'VZ', 'VRTX', 'VFC', 'VTRS', 'V', 'VNO', 'VMC', 'WRB',
            'WAB', 'WMT', 'WBA', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC',
            'WU', 'WRK', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL', 'XYL',
            'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS'
        ]
        return full_sp500
    
    def get_euro_stoxx_50_tickers(self):
        """Get EURO STOXX 50 companies"""
        euro_stoxx_50 = [
            'ASML.AS', 'SAN.PA', 'AIR.PA', 'OR.PA', 'MC.PA', 'SAF.PA', 'AI.PA', 'DG.PA', 
            'BNP.PA', 'ENEL.MI', 'ENI.MI', 'ISP.MI', 'ADS.DE', 'ALV.DE', 'BAS.DE', 
            'BAYN.DE', 'BMW.DE', 'CON.DE', 'DAI.DE', 'DBK.DE', 'DB1.DE', 'DPW.DE', 
            'DTE.DE', 'EOAN.DE', 'FME.DE', 'HEI.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 
            'MRK.DE', 'MUV2.DE', 'RWE.DE', 'SAP.DE', 'SIE.DE', 'VOW3.DE', 'ABI.BR', 
            'INGA.AS', 'PHIA.AS', 'UNA.AS', 'BBVA.MC', 'SAN.MC', 'IBE.MC', 'ITX.MC', 
            'REP.MC', 'NOVOb.CO', 'NESN.SW', 'ROG.SW', 'NOVN.SW', 'UBSG.SW'
        ]
        return euro_stoxx_50
    
    def get_global_tickers(self):
        """Combine S&P 500 and EURO STOXX 50"""
        sp500_tickers = self.get_sp500_tickers()
        euro_tickers = self.get_euro_stoxx_50_tickers()
        
        all_tickers = sp500_tickers + euro_tickers
        
        return list(set(all_tickers))
    
    def get_company_info(self, ticker, max_retries=3):
        """
        Get company information for a single ticker with error handling
        """
        for attempt in range(max_retries):
            try:
                stock = yf.Ticker(ticker)
                info = stock.info
                
                # Extract key information
                company_data = {
                    'ticker': ticker,
                    'name': info.get('longName', ''),
                    'sector': info.get('sector', ''),
                    'industry': info.get('industry', ''),
                    'country': info.get('country', ''),
                    'market_cap': info.get('marketCap', None),
                    'exchange': info.get('exchange', ''),
                    'currency': info.get('currency', ''),
                    'last_updated': pd.Timestamp.now()
                }
                return company_data
                
            except Exception as e:
                if attempt == max_retries - 1:
                    print(f"Failed to get data for {ticker}: {e}")
                    return None
                time.sleep(2)  # Wait before retry
    
    def build_company_database(self, use_popular_list=True, use_global_list=False, max_companies=100):
        """
        Build a database of company information
        """
        if use_global_list:
            tickers = self.get_global_tickers()
        elif use_popular_list:
            tickers = self.get_popular_tickers()
        else:
            tickers = self.get_all_tickers()
        
        # Limit to max_companies for testing
        tickers = tickers[:max_companies]
        
        print(f"Processing {len(tickers)} companies...")
        
        companies_data = []
        for ticker in tqdm(tickers):
            company_info = self.get_company_info(ticker)
            if company_info:
                companies_data.append(company_info)
            time.sleep(0.5)  # Rate limiting
        
        self.companies_df = pd.DataFrame(companies_data)
        print(f"Successfully retrieved data for {len(self.companies_df)} companies")
        return self.companies_df
    
    def save_to_csv(self, filename='yfinance_companies.csv'):
        """Save database to CSV"""
        if self.companies_df is not None:
            self.companies_df.to_csv(filename, index=False)
            print(f"Database saved to {filename}")
        else:
            print("No data to save. Run build_company_database() first.")
    
    def save_to_sqlite(self, db_name='yfinance_companies.db'):
        """Save database to SQLite"""
        if self.companies_df is not None:
            conn = sqlite3.connect(db_name)
            self.companies_df.to_sql('companies', conn, if_exists='replace', index=False)
            conn.close()
            print(f"Database saved to {db_name}")
        else:
            print("No data to save. Run build_company_database() first.")
    
    def get_database_stats(self):
        """Get statistics about the database"""
        if self.companies_df is not None:
            print("Database Statistics:")
            print(f"Total companies: {len(self.companies_df)}")
            print(f"Sectors: {self.companies_df['sector'].nunique()}")
            print(f"Industries: {self.companies_df['industry'].nunique()}")
            print(f"Countries: {self.companies_df['country'].nunique()}")
            print("\nSector distribution:")
            print(self.companies_df['sector'].value_counts())
        else:
            print("No database available. Run build_company_database() first.")

# Updated usage examples:
def quick_setup_global():
    """Quick setup for global companies (S&P 500 + EURO STOXX 50)"""
    db = YFinanceCompanyDatabase()
    
    # Build database with global companies - set max_companies high enough to get all
    companies_df = db.build_company_database(use_popular_list=False, use_global_list=True, max_companies=600)
    
    # Save to CSV for analysis
    db.save_to_csv('global_stock_companies.csv')
    
    # Show statistics
    db.get_database_stats()
    
    return db, companies_df
    
# Run this in your Jupyter notebook for GLOBAL companies:
db, companies_df = quick_setup_global()


Processing 552 companies...


100%|██████████| 552/552 [10:49<00:00,  1.18s/it]

Successfully retrieved data for 552 companies
Database saved to global_stock_companies.csv
Database Statistics:
Total companies: 552
Sectors: 12
Industries: 114
Countries: 14

Sector distribution:
sector
Technology                85
Financial Services        77
Industrials               75
Healthcare                68
Consumer Cyclical         62
Consumer Defensive        41
Utilities                 35
Real Estate               31
Communication Services    25
Energy                    24
Basic Materials           23
                           6
Name: count, dtype: int64


In [2]:
db.get_database_stats()

Database Statistics:
Total companies: 552
Sectors: 12
Industries: 114
Countries: 14

Sector distribution:
sector
Technology                85
Financial Services        77
Industrials               75
Healthcare                68
Consumer Cyclical         62
Consumer Defensive        41
Utilities                 35
Real Estate               31
Communication Services    25
Energy                    24
Basic Materials           23
                           6
Name: count, dtype: int64


In [3]:
companies_df.head()

,ticker,name,sector,industry,country,market_cap,exchange,currency,last_updated
0,FDS,FactSet Research Systems Inc.,Financial Services,Financial Data & Stock Exchanges,United States,1.069588e+10,NYQ,USD,2025-10-16 17:06:42.075720
1,ITX.MC,"Industria de Diseño Textil, S.A.",Consumer Cyclical,Apparel Retail,Spain,1.498661e+11,MCE,EUR,2025-10-16 17:06:43.158043
2,NOW,"ServiceNow, Inc.",Technology,Software - Application,United States,1.901465e+11,NYQ,USD,2025-10-16 17:06:44.159094
3,TYL,"Tyler Technologies, Inc.",Technology,Software - Application,United States,2.167633e+10,NYQ,USD,2025-10-16 17:06:45.336030
4,AXP,American Express Company,Financial Services,Credit Services,United States,2.284512e+11,NYQ,USD,2025-10-16 17:06:46.392869
